##### Formulation of the GAMS model in JuMP from 
###### AN IMPROVED FORMULATION FOR A GAS TRANSPORTATION PROBLEM  (ERWIN KALVELAGEN)
###### https://www.researchgate.net/publication/2568197_An_Improved_Formulation_For_A_Gas_Transportation_Problem

##### Further information on JuMP: 
https://arxiv.org/pdf/1508.01982.pdf

# Read data and Data cleaning

In [1]:
using DataFrames
using JuMP
#using ExcelReaders

In [2]:
Ndata = readtable("Ndata.csv", separator=';') # node data
head(Ndata)

,x,slo,sup,plo,pup,c
1,Anderlues,0.0,1.2,0,66.2,1.68
2,Antwerpen,-Inf,-4.034,30,80.0,0.0
3,Arlon,-Inf,-0.222,0,66.2,0.0
4,Berneau,0.0,0.0,0,66.2,0.0
5,Blaregnies,-Inf,-15.616,50,66.2,0.0
6,Brugge,-Inf,-3.918,30,80.0,0.0


In [3]:
Adata = readtable("Adata.csv", separator=';') # line data
head(Adata)

,a,i,j,D,L,act
1,1,Zeebrugge,Dudzele,890.0,4.0,0
2,2,Zeebrugge,Dudzele,890.0,4.0,0
3,3,Dudzele,Brugge,890.0,6.0,0
4,4,Dudzele,Brugge,890.0,6.0,0
5,5,Brugge,Zomergem,890.0,26.0,0
6,6,Loenhout,Antwerpen,590.1,43.0,0


## Constants

In [6]:
T = 281.5
z = 0.8
epsilon = 0.05
delta = 0.6106;

## Read edge data
### Calculate pipeline characteristics

$$ lambda_{aij} = \frac {1} { \frac {\sqrt{2* \log_{10}*3.7*diameter_{aij}}} {epsilon}}  $$

$$ pipecoeff_{aij} = 96.074830*10^{-15} * \frac {diameter_{aij}^5 } { lambda_{aij}*z*T*length_{aij}*delta) } $$

### Define gas pipeline network as graph with edges 

Network is respected as a Graph $G (N,E) $ with N set of nodes and the set of edges $E \subseteq N x N $ 


$e_{a,i,j}$ reprents one arc a from node i to node j

Because there exists lines from the same node i to node j, the set a is necessary
###### JuMP implementation following partly https://arxiv.org/pdf/1508.01982.pdf p.6



In [7]:
head(Adata)

,a,i,j,D,L,act
1,1,Zeebrugge,Dudzele,890.0,4.0,0
2,2,Zeebrugge,Dudzele,890.0,4.0,0
3,3,Dudzele,Brugge,890.0,6.0,0
4,4,Dudzele,Brugge,890.0,6.0,0
5,5,Brugge,Zomergem,890.0,26.0,0
6,6,Loenhout,Antwerpen,590.1,43.0,0


formulas from
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.14.5465&rep=rep1&type=pdf

In [8]:
df = Adata

immutable Edge
    route; from; to; pipecoeff; active
end

edges = []
active_edges = []
passive_edges = []

for itera in 1:size(df, 1)
    a = df[itera,:1] #set route
    i = df[itera,:2] #set start node
    j = df[itera,:3] #set end node
    d = df[itera,:4] #diameter
    l = df[itera,:5] #length
    act = df[itera,:6] #active? --> compressor
    lambda = 1/(2*log10(3.7*d/epsilon))^2 #
    pipecoeff = 96.074830e-15 *  (d^5/(lambda*z*T*l*delta) )

    push!(edges,Edge(a,i,j,pipecoeff,act))
    
    if act==1
        push!(active_edges,Edge(a,i,j,pipecoeff,act))
    else
        push!(passive_edges,Edge(a,i,j,pipecoeff,act))
    end
end
edges

24-element Array{Any,1}:
 Edge(1, "Zeebrugge", "Dudzele", 9.059, 0)    
 Edge(2, "Zeebrugge", "Dudzele", 9.059, 0)    
 Edge(3, "Dudzele", "Brugge", 6.03933, 0)     
 Edge(4, "Dudzele", "Brugge", 6.03933, 0)     
 Edge(5, "Brugge", "Zomergem", 1.39369, 0)    
 Edge(6, "Loenhout", "Antwerpen", 0.100131, 0)
 Edge(7, "Antwerpen", "Gent", 0.14847, 0)     
 Edge(8, "Gent", "Zomergem", 0.226613, 0)     
 Edge(9, "Zomergem", "Peronnes", 0.658836, 0) 
 Edge(10, "Voeren", "Berneau", 7.2472, 1)     
 Edge(11, "Voeren", "Berneau", 0.107192, 1)   
 Edge(12, "Berneau", "Liege", 1.8118, 0)      
 Edge(13, "Berneau", "Liege", 0.0267981, 0)   
 Edge(14, "Liege", "Warnand", 1.44944, 0)     
 Edge(15, "Liege", "Warnand", 0.0214385, 0)   
 Edge(16, "Warnand", "Namur", 0.862762, 0)    
 Edge(17, "Namur", "Anderlues", 0.9059, 0)    
 Edge(18, "Anderlues", "Peronnes", 7.2472, 0) 
 Edge(19, "Peronnes", "Mons", 3.6236, 0)      
 Edge(20, "Mons", "Blaregnies", 1.44944, 0)   
 Edge(21, "Warnand", "Wanze", 0.051

## Read node data

In [10]:
active_edges

3-element Array{Any,1}:
 Edge(10, "Voeren", "Berneau", 7.2472, 1)  
 Edge(11, "Voeren", "Berneau", 0.107192, 1)
 Edge(22, "Wanze", "Sinsin", 0.00641179, 1)

In [11]:
df = Ndata

immutable Node
    i; lowerSupply; upperSupply; lowerPressure; upperPressure; cost
end

nodes = Dict()

for itera in 1:size(df, 1)
    i = df[itera,:1]   #set node
    slo = df[itera,:2] #set lowerSupply
    sup = df[itera,:3] #set upperSupply
    plo = df[itera,:4] #set lowerPressure
    pup = df[itera,:5] #set upperPressure
    c = df[itera,:6] #set cost
    
    nodes[i] = Node(i,slo,sup,plo,pup,c)
end

# Model 

In [12]:
using Gurobi # Solver

In [358]:
using Ipopt

In [376]:
#m = Model(solver = GurobiSolver(Presolve=0)) # Create model
m = Model(solver = IpoptSolver()) # Create model

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is Ipopt

In [377]:
using CoinOptServices

#m = Model(solver = OsilSolver()) # Create model



## Variables

##### When using a tuple as an index (a,i,j) the automated latex generation does not  work properly

In [378]:
edges

24-element Array{Any,1}:
 Edge(1, "Zeebrugge", "Dudzele", 9.059, 0)    
 Edge(2, "Zeebrugge", "Dudzele", 9.059, 0)    
 Edge(3, "Dudzele", "Brugge", 6.03933, 0)     
 Edge(4, "Dudzele", "Brugge", 6.03933, 0)     
 Edge(5, "Brugge", "Zomergem", 1.39369, 0)    
 Edge(6, "Loenhout", "Antwerpen", 0.100131, 0)
 Edge(7, "Antwerpen", "Gent", 0.14847, 0)     
 Edge(8, "Gent", "Zomergem", 0.226613, 0)     
 Edge(9, "Zomergem", "Peronnes", 0.658836, 0) 
 Edge(10, "Voeren", "Berneau", 7.2472, 1)     
 Edge(11, "Voeren", "Berneau", 0.107192, 1)   
 Edge(12, "Berneau", "Liege", 1.8118, 0)      
 Edge(13, "Berneau", "Liege", 0.0267981, 0)   
 Edge(14, "Liege", "Warnand", 1.44944, 0)     
 Edge(15, "Liege", "Warnand", 0.0214385, 0)   
 Edge(16, "Warnand", "Namur", 0.862762, 0)    
 Edge(17, "Namur", "Anderlues", 0.9059, 0)    
 Edge(18, "Anderlues", "Peronnes", 7.2472, 0) 
 Edge(19, "Peronnes", "Mons", 3.6236, 0)      
 Edge(20, "Mons", "Blaregnies", 1.44944, 0)   
 Edge(21, "Warnand", "Wanze", 0.051

In [379]:
# flow variable

@variable(m,  flow[aij in edges] )

flow[aij] ∀ aij ∈ {Edge(1, "Zeebrugge", "Dudzele", 9.058997518021224, 0),Edge(2, "Zeebrugge", "Dudzele", 9.058997518021224, 0),…,Edge(23, "Sinsin", "Arlon", 0.0017010864366246608, 0),Edge(24, "Arlon", "Petange", 0.027784411798202797, 0)}

In [380]:
# supply variable

@variable(m, nodes[node].lowerSupply  <= s[node in keys(nodes)]  <= nodes[node].upperSupply)

… ≤ s[node] ≤ … ∀ node ∈ {Gent,Sinsin,…,Berneau,Wanze}

In [381]:
m

Feasibility problem with:
 * 0 linear constraints
 * 44 variables
Solver is Ipopt

In [382]:
# squared pressure--> linearization

@variable(m, (nodes[node].lowerPressure)^2 <= pi[node in keys(nodes)] <= 
    (nodes[node].upperPressure)^2 ) 

… ≤ pi[node] ≤ … ∀ node ∈ {Gent,Sinsin,…,Berneau,Wanze}

## Constraints

In [383]:
I = unique([edge.to for edge in edges])  ;

In [384]:
# (1)
@constraint(m,flowcon[node = I], sum{flow[e], e in edges; e.from == node}
  == sum{flow[e] , e in edges; e.to == node} + s[node]);

In [385]:
# (2)
@constraint(m,freevar[node = active_edges], flow[node] >= 0 );

In [386]:
T

281.5

In [387]:
# Linear approximations of the flow

# (2) passive
#@constraint(m,passiveflow[node = passive_edges], flow[node] ==  node.pipecoeff * (pi[node.from]-pi[node.to]));

# (3) active
@constraint(m,activeflow[node = active_edges], flow[node] >=  node.pipecoeff * (pi[node.from]-pi[node.to]));


## Objective

In [388]:
@objective(m, Min, sum{node.cost * s[node.i], node in values(nodes)})

1.68 s[Anderlues] + 2.28 s[Zeebrugge] + 1.68 s[Voeren] + 2.28 s[Dudzele] + 2.28 s[Loenhout] + 1.68 s[Peronnes]

## Solve

In [389]:
include("gurobi_iis.jl") #module for calculating IIS in case of infeasability

status = solve(m)

if string(status) == "Infeasible"
    print_iis_gurobi(m)
end

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       55
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       59
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       26
                     variables with only upper bounds:        9
Total number of equality constraints.................:       17
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [390]:
println("pi = ", getvalue(flow))

pi = flow: 1 dimensions:
[   Edge(1, "Zeebrugge", "Dudzele", 9.058997518021224, 0)] = 1637.6520472525324
[   Edge(2, "Zeebrugge", "Dudzele", 9.058997518021224, 0)] = 1637.6578638644576
[      Edge(3, "Dudzele", "Brugge", 6.039331678680816, 0)] = 1637.6549555540462
[      Edge(4, "Dudzele", "Brugge", 6.039331678680816, 0)] = 1637.6549555540432
[    Edge(5, "Brugge", "Zomergem", 1.3936919258494191, 0)] = 2083.4376903057146
[Edge(6, "Loenhout", "Antwerpen", 0.10013120911442538, 0)] = 2944.828517083997
[    Edge(7, "Antwerpen", "Gent", 0.14847041351449283, 0)] = 1755.2774953312817
[     Edge(8, "Gent", "Zomergem", 0.22661273641685747, 0)] = 658.647419609713
[  Edge(9, "Zomergem", "Peronnes", 0.6588361831288163, 0)] = 2742.085109915428
[      Edge(10, "Voeren", "Berneau", 7.24719801441698, 1)] = 1562.3717824976304
[   Edge(11, "Voeren", "Berneau", 0.10719241288031257, 1)] = 1357.6346849804381
[      Edge(12, "Berneau", "Liege", 1.811799503604245, 0)] = 1460.0032337390414
[   Edge(13, "Berne

## Add non linearities

In [391]:
# Non linearities

#@variable(m, z)


@variable(m,  z[aij in edges]  )

@constraint(m, slackposri[node = edges], z[node] >= flow[node] );
@constraint(m, slackposle[node = edges], z[node] >= -flow[node] );


#@constraint(m, z[aij] >= flow)
#@constraint(m, z >= -flow)

# (2) passive
@constraint(m,passiveflow[node = passive_edges], z[node]*flow[node] ==  node.pipecoeff * (pi[node.from]-pi[node.to]));

# (3) active
@constraint(m,activeflow[node = active_edges], flow[node]*flow[node] >=  node.pipecoeff * (pi[node.from]-pi[node.to]));



In [392]:
include("gurobi_iis.jl") #module for calculating IIS in case of infeasability
status = solve(m)


This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      139
Number of nonzeros in inequality constraint Jacobian.:      120
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:       83
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       26
                     variables with only upper bounds:        9
Total number of equality constraints.................:       38
Total number of inequality constraints...............:       57
        inequality constraints with only lower bounds:       57
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

# Evaluation

In [395]:
println("flow = ", getvalue(flow))

flow = flow: 1 dimensions:
[   Edge(1, "Zeebrugge", "Dudzele", 9.058997518021224, 0)] = 11.697686274823408
[   Edge(2, "Zeebrugge", "Dudzele", 9.058997518021224, 0)] = 11.69767448366753
[      Edge(3, "Dudzele", "Brugge", 6.039331678680816, 0)] = 11.697680374795016
[      Edge(4, "Dudzele", "Brugge", 6.039331678680816, 0)] = 11.697680374795015
[    Edge(5, "Brugge", "Zomergem", 1.3936919258494191, 0)] = 15.850524564255089
[Edge(6, "Loenhout", "Antwerpen", 0.10013120911442538, 0)] = 7.8634947523676155
[    Edge(7, "Antwerpen", "Gent", 0.14847041351449283, 0)] = 3.384712417881812
[     Edge(8, "Gent", "Zomergem", 0.22661273641685747, 0)] = -2.19453462599948
[  Edge(9, "Zomergem", "Peronnes", 0.6588361831288163, 0)] = 13.655989938255608
[      Edge(10, "Voeren", "Berneau", 7.24719801441698, 1)] = 3.19402785097039
[   Edge(11, "Voeren", "Berneau", 0.10719241288031257, 1)] = 25.848611127172447
[      Edge(12, "Berneau", "Liege", 1.811799503604245, 0)] = 28.97163151379611
[   Edge(13, "Berne